In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pathlib
import numpy as np

In [ ]:
#Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_classes = 3
learning_rate = 0.001


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def loader(path):
    sample = torch.load(path)
    return sample
    

In [ ]:
train_path= '/content/drive/My Drive/minor_project_latent/train/blurs'
test_path= '/content/drive/My Drive/minor_project_latent/test/blurs'

train_dataset = torchvision.datasets.DatasetFolder(root=train_path, loader=loader,extensions='.txt')

train_loader=DataLoader(train_dataset, worker_init_fn=np.random.seed(0))

test_dataset = torchvision.datasets.DatasetFolder(root=test_path,loader=loader,extensions='.txt')

test_loader=DataLoader(test_dataset, worker_init_fn=np.random.seed(0))


In [ ]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['average', 'bad', 'good']


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self,   num_classes):
        super(NeuralNet, self).__init__()

        self.conv1 = nn.Conv2d(81, 86, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(86, 91, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(91, 96, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(4*4*96 , 308 )
        self.fc2 = nn.Linear(308 , 60)
        self.fc3 = nn.Linear(60 , num_classes)

        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.conv1(x)     #32*32*86
        out = self.relu(out) 
        out = self.pool1(out)   #16*16*86
        out = self.relu(out)

        out = self.conv3(out)     #16*16*91
        out = self.relu(out)
        out = self.pool2(out)     #8*8*91
        out = self.relu(out)

        out = self.conv5(out)     #8*8*96
        out = self.relu(out) 
        out = self.pool2(out)     #4*4*96
        out = self.relu(out) 

        out = out.view(-1, 4 * 4 * 96) #1536
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [ ]:
model = NeuralNet( num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
#criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


In [ ]:
num_epochs = 1

# Train the model

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        images = images.squeeze(1)

        # Forward pass
        outputs = model(images)

        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 120 == 0:
          print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


In [ ]:
'''FILE = "/content/drive/My Drive/model_save/pytorch_model.h5"
torch.save(model, FILE)
loaded_model=torch.load(FILE)
loaded_model.eval()
print(loaded_model.parameters())
'''

'FILE = "/content/drive/My Drive/model_save/pytorch_model.h5"\ntorch.save(model, FILE)\nloaded_model=torch.load(FILE)\nloaded_model.eval()\nprint(loaded_model.parameters())\n'

In [ ]:
# Test the model
model.eval()

class_correct = list(0. for i in range(3))
class_total = list(0. for i in range(3))

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        images=images.squeeze(1)
        #print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(1)))
        outputs = model(images)

        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        #print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(1)))

        c = (predicted == labels).squeeze()

        #label = labels[i]
        #print("B",class_correct[label])
        #print("c",label.item())
        #print("d",c)
        class_correct[labels] += c
        class_total[labels] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on test images: {acc} %\n')

    for i in range(3):
      print('Accuracy of class %2s : %.4f %%' % (classes[i], 100 * class_correct[i] / class_total[i])) 

Accuracy of the network on test images: 33.333333333333336 %

Accuracy of class average : 0.0000 %
Accuracy of class bad : 0.0000 %
Accuracy of class good : 100.0000 %
